In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input, Average
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy as scc
from tensorflow.keras.datasets import mnist
from tensorflow.keras.regularizers import l2
from spectraltools import Spectral

# Example of spectral training

In the following a branched functional model is created using several Spectral Layers. An L2 regularization is also applied as we would like to prune the model later on. The model is trained for 10 epoch and then evaluated on the test set. 

In [20]:
# Dataset and model creation
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.
x_test = x_test / 255.

spectral_configuration = {'activation': 'relu', 
                          'use_bias': True,
                          'base_regularizer': l2(1E-3),
                          'diag_regularizer': l2(5E-3)}

inputs = Input(shape=(28, 28,))
x = Flatten()(inputs)
y = Spectral(200,  **spectral_configuration, name='Spec1')(x)
y = Spectral(300,  **spectral_configuration, name='Spec2')(y)
y = Spectral(300,  **spectral_configuration, name='Dense1')(y)

x = Spectral(200, **spectral_configuration, name='Spec3')(x)
x = Spectral(300, **spectral_configuration, name='Spec4')(x)
x = Spectral(300,**spectral_configuration, name='Spec5')(x)

z = Average()([x, y])
outputs = Dense(10, activation="softmax", name='LastDense')(z)

model = Model(inputs=inputs, outputs=outputs, name="branched")

model.compile(optimizer=Adam(1E-3), loss=scc(from_logits=False), metrics=["accuracy"])
model.fit(x_train, y_train, validation_split=0.2, batch_size=300, epochs=10, verbose=1)
model.evaluate(x_test, y_test, batch_size=300)

Epoch 1/10
160/160 [==============================] - 4s 18ms/step - loss: 8.1912 - accuracy: 0.8869 - val_loss: 6.5784 - val_accuracy: 0.9449
Epoch 2/10
160/160 [==============================] - 3s 16ms/step - loss: 5.5607 - accuracy: 0.9433 - val_loss: 4.6209 - val_accuracy: 0.9538
Epoch 3/10
160/160 [==============================] - 3s 16ms/step - loss: 3.9145 - accuracy: 0.9507 - val_loss: 3.2709 - val_accuracy: 0.9539
Epoch 4/10
160/160 [==============================] - 3s 17ms/step - loss: 2.7920 - accuracy: 0.9527 - val_loss: 2.3576 - val_accuracy: 0.9565
Epoch 5/10
160/160 [==============================] - 2s 12ms/step - loss: 2.0541 - accuracy: 0.9504 - val_loss: 1.7692 - val_accuracy: 0.9553
Epoch 6/10
160/160 [==============================] - 2s 11ms/step - loss: 1.5809 - accuracy: 0.9503 - val_loss: 1.4014 - val_accuracy: 0.9526
Epoch 7/10
160/160 [==============================] - 2s 12ms/step - loss: 1.2761 - accuracy: 0.9482 - val_loss: 1.1485 - val_accuracy: 0.9518

[0.6774107217788696, 0.9538000226020813]

# Example of spectral pruning
Now that the model has been trained, we can prune it. In the following we will prune the 30% of the spectral layers nodes according to their relevance. The model is then evaluated on the test set.

In [21]:
from spectraltools import prune_percentile, metric_based_pruning
from spectraltools.spectralprune import original_model

In [22]:
# Now the 30% of the spectral layers node will be in place pruned according to their relevance. The eigenvalues whose magnitude is smaller than the corresponding percentile will be set to zero by masking the corresponding weights. This will also have an effect on the corresponding bias which will be also masked.
prune_percentile(model, 30)
print(f'Pruned accuracy: {model.evaluate(x_test, y_test, batch_size=300)[1]:.3f}')

Number of nodes masked: 480 out of 1600 (30.00%)
34/34 [==============================] - 1s 9ms/step - loss: 0.6834 - accuracy: 0.9530
Pruned accuracy: 0.953


As we can see masking 30% of the eigenvalues has basically no impact in the accuracy of the model. This is due to the fact that the pruned eigenvalues are very small and therefore their contribution to the model is negligible; making the whole feature not relevant.


# Example of metric based spectral pruning 
In the following code we will prune the model according to the metric based approach. In this case we will prune until a given drop in the accuracy is reached. In this case we will prune until the accuracy drops by 5%.

In [23]:
# Reset the model to the unpruned state: the mask are all set to None
original_model(model)

metric_based_pruning(model, 
                     eval_dictionary=dict(x=x_train, y=y_train, batch_size=300),
                      compare_metric='accuracy',
                      max_delta_percent=5)

200/200 [==============================] - 2s 8ms/step - loss: 0.6802 - accuracy: 0.9562
Number of nodes masked: 0 out of 1600 (0.00%)
200/200 [==============================] - 2s 8ms/step - loss: 0.6737 - accuracy: 0.9581
Pruning with 0% of eigenvalues removed. Delta in accuracy: 0.20%
Number of nodes masked: 80 out of 1600 (5.00%)
200/200 [==============================] - 2s 6ms/step - loss: 0.6737 - accuracy: 0.9581
Pruning with 5% of eigenvalues removed. Delta in accuracy: 0.20%
Number of nodes masked: 160 out of 1600 (10.00%)
200/200 [==============================] - 2s 7ms/step - loss: 0.6737 - accuracy: 0.9581
Pruning with 10% of eigenvalues removed. Delta in accuracy: 0.20%
Number of nodes masked: 240 out of 1600 (15.00%)


KeyboardInterrupt: 